In [7]:
pip install python-sat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Model.py

In [8]:
%%writefile model.py
class SudokuGrid:
    """
    Represents the Sudoku Board state.
    """
    def __init__(self, matrix=None):
        # 0 represents an empty cell
        if matrix:
            self.grid = matrix
        else:
            self.grid = [[0 for _ in range(9)] for _ in range(9)]

    def __str__(self):
        return str(self.grid)

class VariableMapper:
    """
    Handles mapping between Sudoku logic (row, col, value)
    and SAT variable IDs (1 to 729).

    Variable ID = (row * 81) + (col * 9) + (val - 1) + 1
    """
    @staticmethod
    def to_var(r, c, v):
        """
        r: 0-8 (row)
        c: 0-8 (col)
        v: 1-9 (value)
        Returns: Integer ID >= 1
        """
        return (r * 9 + c) * 9 + (v - 1) + 1

    @staticmethod
    def to_rcv(var_id):
        """
        Returns (row, col, val) tuple from variable ID.
        """
        adjusted = var_id - 1
        val = (adjusted % 9) + 1
        c = (adjusted // 9) % 9
        r = (adjusted // 81)
        return r, c, val

Overwriting model.py


# Problem.py

In [9]:
%%writefile problem.py
from model import VariableMapper
import itertools

class SudokuClauseGenerator:
    """
    Generates CNF clauses for the Sudoku CSP.
    """
    def __init__(self):
        self.clauses = []

    def _add_cell_constraints(self):
        """
        1. Definedness: Each cell has at least one value (1-9).
        2. Uniqueness: Each cell has at most one value.
        """
        for r in range(9):
            for c in range(9):
                # Definedness: (X_rc1 v X_rc2 ... v X_rc9)
                self.clauses.append([VariableMapper.to_var(r, c, v) for v in range(1, 10)])

                # Uniqueness: (~X_rcj v ~X_rck) for j != k
                # If cell has val j, it cannot have val k
                for j in range(1, 10):
                    for k in range(j + 1, 10):
                        self.clauses.append([
                            -VariableMapper.to_var(r, c, j),
                            -VariableMapper.to_var(r, c, k)
                        ])

    def _add_line_constraints(self):
        """
        Each value (1-9) appears at most once in each Row and Column.
        """
        for v in range(1, 10):
            # Row constraints
            for r in range(9):
                for c1 in range(9):
                    for c2 in range(c1 + 1, 9):
                        self.clauses.append([
                            -VariableMapper.to_var(r, c1, v),
                            -VariableMapper.to_var(r, c2, v)
                        ])

            # Column constraints
            for c in range(9):
                for r1 in range(9):
                    for r2 in range(r1 + 1, 9):
                        self.clauses.append([
                            -VariableMapper.to_var(r1, c, v),
                            -VariableMapper.to_var(r2, c, v)
                        ])

    def _add_box_constraints(self):
        """
        Each value (1-9) appears at most once in each 3x3 Box.
        """
        for v in range(1, 10):
            for box_r in range(0, 9, 3):
                for box_c in range(0, 9, 3):
                    # Collect cells in this 3x3 box
                    cells = []
                    for r in range(3):
                        for c in range(3):
                            cells.append((box_r + r, box_c + c))

                    # Pairwise constraints within box
                    for i in range(len(cells)):
                        for j in range(i + 1, len(cells)):
                            r1, c1 = cells[i]
                            r2, c2 = cells[j]
                            self.clauses.append([
                                -VariableMapper.to_var(r1, c1, v),
                                -VariableMapper.to_var(r2, c2, v)
                            ])

    def _add_prefilled_constraints(self, grid):
        """
        Enforces the constraints provided by the initial puzzle state.
        Clause: (X_rcv) must be True.
        """
        for r in range(9):
            for c in range(9):
                val = grid.grid[r][c]
                if val != 0:
                    self.clauses.append([VariableMapper.to_var(r, c, val)])

    def get_cnf(self, initial_grid):
        """
        Generates all constraints and returns list of clauses.
        """
        self.clauses = [] # Reset
        self._add_cell_constraints()
        self._add_line_constraints()
        self._add_box_constraints()
        self._add_prefilled_constraints(initial_grid)
        return self.clauses

Overwriting problem.py


# Search.py

In [10]:
%%writefile search.py
from pysat.solvers import Glucose3
from model import SudokuGrid, VariableMapper
from problem import SudokuClauseGenerator

class SudokuAgent:
    """
    The Agent that takes a Sudoku matrix and returns a solution.
    """
    def solve(self, matrix):
        # 1. Create Grid Model
        input_grid = SudokuGrid(matrix)

        # 2. Generate CSP/CNF Clauses
        generator = SudokuClauseGenerator()
        clauses = generator.get_cnf(input_grid)

        # 3. Initialize Solver (Glucose3)
        solver = Glucose3()
        for clause in clauses:
            solver.add_clause(clause)

        # 4. Solve
        is_satisfiable = solver.solve()

        if not is_satisfiable:
            print("No solution found.")
            return None

        # 5. Extract Model
        model_vars = solver.get_model()

        # 6. Convert SAT Model back to Sudoku Grid
        solution_matrix = [[0]*9 for _ in range(9)]

        for var_id in model_vars:
            if var_id > 0: # Only True variables matter
                r, c, v = VariableMapper.to_rcv(var_id)
                solution_matrix[r][c] = v

        solver.delete()
        return SudokuGrid(solution_matrix)

Overwriting search.py


# Utils.py

In [11]:
%%writefile utils.py
class Visualizer:
    @staticmethod
    def display(grid_obj):
        if not grid_obj:
            print("No grid to display.")
            return

        board = grid_obj.grid
        print("-" * 25)
        for i in range(9):
            line = "| "
            for j in range(9):
                val = board[i][j]
                line += (str(val) if val != 0 else ".") + " "
                if (j + 1) % 3 == 0:
                    line += "| "
            print(line)
            if (i + 1) % 3 == 0:
                print("-" * 25)

Overwriting utils.py


# Main.py

In [12]:
import sys
import time
# Ensure python can find the files we just wrote
sys.path.append('.')

from search import SudokuAgent
from utils import Visualizer

# The Sudoku matrix from the user's image
# 0 denotes empty cells
puzzle_input = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

def main():
    print(">>> Initial Sudoku Puzzle:")
    # Temporary object just for display
    from model import SudokuGrid
    Visualizer.display(SudokuGrid(puzzle_input))

    print("\n>>> Initializing Agent and Generating Constraints...")
    agent = SudokuAgent()

    start_time = time.time()
    result_grid = agent.solve(puzzle_input)
    end_time = time.time()

    if result_grid:
        print(f"\n>>> Solution Found! (Time: {end_time - start_time:.4f}s)")
        Visualizer.display(result_grid)
    else:
        print("\n>>> Failed to find a solution.")

if __name__ == "__main__":
    main()

>>> Initial Sudoku Puzzle:
-------------------------
| 5 3 . | . 7 . | . . . | 
| 6 . . | 1 9 5 | . . . | 
| . 9 8 | . . . | . 6 . | 
-------------------------
| 8 . . | . 6 . | . . 3 | 
| 4 . . | 8 . 3 | . . 1 | 
| 7 . . | . 2 . | . . 6 | 
-------------------------
| . 6 . | . . . | 2 8 . | 
| . . . | 4 1 9 | . . 5 | 
| . . . | . 8 . | . 7 9 | 
-------------------------

>>> Initializing Agent and Generating Constraints...

>>> Solution Found! (Time: 0.0106s)
-------------------------
| 5 3 4 | 6 7 8 | 9 1 2 | 
| 6 7 2 | 1 9 5 | 3 4 8 | 
| 1 9 8 | 3 4 2 | 5 6 7 | 
-------------------------
| 8 5 9 | 7 6 1 | 4 2 3 | 
| 4 2 6 | 8 5 3 | 7 9 1 | 
| 7 1 3 | 9 2 4 | 8 5 6 | 
-------------------------
| 9 6 1 | 5 3 7 | 2 8 4 | 
| 2 8 7 | 4 1 9 | 6 3 5 | 
| 3 4 5 | 2 8 6 | 1 7 9 | 
-------------------------
